<a href="https://colab.research.google.com/github/lunadunkel/ML-MSU-projects/blob/main/course-word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Предобработка

Для нормального распознавания по предложениям я решила привести все сразу в относительно унифицированный вид: так как тексты новостные, там присутствуют заголовки без точек (с двумя переносами строк) – их я привела к виду предложений с точкой. То есть покрытие все-таки будет меньше, чем 93%, так как sent_tokenize нлтк и sentencizer спейси эти случаи с заголовками считал одним предложением (upd.: 92 в пересчете). Кажется все-таки логичным их считать отдельными предложениями скорее. Дальше надо избавиться от всех конечных источников с ссылками, так как они будут только захламлять и мешать. Дальше обычные '\n' были заменены пока что на один пробел, и этот пробел учитывается в предложениях, иначе чаровая индексация полетит; как сделать это более экономно я понимаю (очевидно все на символ назад смещать каждый раз), но я боюсь, что полетит индексация, а с ней итак очень много возиться приходится.

In [ ]:
!curl -L -o train  "drive.google.com/uc?export=download&id=14EmZaMcz2VY2AgzakD1u4N78EleCFINL&confirm=yes"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 7936k  100 7936k    0     0   386k      0  0:00:20  0:00:20 --:--:-- 2551k


In [ ]:
!python -m spacy download ru_core_news_sm
import spacy

tagger = spacy.load("ru_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import json
import os
import re
import numpy as np
import pandas as pd
from collections import defaultdict
from zipfile import ZipFile
from pathlib import Path
from google.colab import files
# from nltk.tokenize import sent_tokenize

In [ ]:
with ZipFile("/content/train", 'r') as file:
  file.extractall(path="/content")
file.close()

In [ ]:
# ни sentencizer у spacy, ни sent_tokenize нлтк мне не нравится, потому будут мои убогие регулярки для чистки текста
bos_pattern = re.compile('(?<!\.)\n\n')
# new_sent_pattern = re.compile('.*[\.\!\?] ')
def clean_data(text):
    text = re.sub(bos_pattern, '. ', text)
    text = re.sub('\n*Источник: .*\n*$', '', text)
    text = re.sub('\n', ' ', text)
    return re.split('(?<=[^\s][.!\?])(?=\s)', text)

In [ ]:
data = []
spans = []

dir = '/content/new_data'

for pos, filename in enumerate(os.listdir(dir)):
    if filename.endswith('.json'):
        f = json.load(open(dir+'/'+filename))
        data.append(f['Text'])
        spans.append(f['Spans'])

df = pd.DataFrame(data=[data, spans], index=['Text', 'Span']).T
dropped_df = df[df['Span'].map(lambda x: bool(x))]

Проблема, которую я обнаружила, но не смогла придумать, как починить: со всякими "кв. м" деление на новые предложения; нлтк и спейси тоже не спасают от этого, но у них еще и пробелы, необходимые для чаровой разметки, убираются, поэтому я оставила свой код.

In [ ]:
overall_spans = 0
ill_span_dist_2 = 0
span_dist_1 = 0
span_dist_2 = 0
ill_texts = defaultdict(list)
ill_distances = defaultdict(list)
# idx_to_dist_2 = defaultdict(dict)
dist_to_idx_2 = defaultdict(dict)
all_dists_2 = set()

for (idx, text, spans) in dropped_df.itertuples():
    new_text = clean_data(text)
    dropped_df.loc[idx]['Text'] = new_text
    for pair in spans:
      overall_spans += 1
      fragment = text[pair[0][0]:pair[1][1]]

      distance = len(clean_data(fragment))
      all_dists_2.add(distance)
      dist_to_idx_2[distance][idx] = pair
      # idx_to_dist_2[idx] = distance

      if distance > 2:
        ill_span_dist_2 += 1
        ill_texts[idx].append(pair)
        ill_distances[idx].append(distance)

      elif distance == 1:
        span_dist_1 += 1

      elif distance == 2:
        span_dist_2 += 1

      else:
        print('?')

In [ ]:
print(f'Дистанция одно предложение: {span_dist_1}')
print(f'Дистанция два предложения: {span_dist_2}')
print(f'Всего предложений: {overall_spans}')
print((span_dist_1+span_dist_2)/overall_spans)

Дистанция одно предложение: 18861
Дистанция два предложения: 8430
Всего предложений: 29655
0.9202832574607992


In [ ]:
dropped_df.to_csv('df_csv')

In [ ]:
def morph_analysis(text, corr_feats, positions):
    true_cand = text[positions[0]:positions[1]]
    doc = tagger(text)
    # print(doc, len(doc))
    candidates = []
    true_index = -1
    cand_added = False
    context = None
    for pos, word in enumerate(doc):
        if str(word) == '-':
            continue

        # print(word, str(word) != '-')
        if true_cand.find(str(word)) != -1:
            if cand_added:
                continue
            true_index = len(candidates)
            candidates.append(positions)
            cand_added = True
            # print(cand_added)
            continue

        if word.tag_ == 'NOUN' and str(word) != '-':
            # print(word)
            # print()
            if ((corr_feats['Gender'] is None or corr_feats['Gender'] == word.morph.get('Gender'))
            and (corr_feats['Number'] is None or corr_feats['Number'] == word.morph.get('Number'))):


              # print(doc[:word.i], len(str(doc[:word.i])), len(str(doc[:word.i+1])))
              begin = word.idx
              end = word.idx+len(word)
              if begin == end:
                  print('Warning: span is empty')
              candidates.append([begin, end])
        context = word
              # print("CAND:", word, f'[{begin}, {end}]')
    if not cand_added:
        true_index = len(candidates)
        candidates.append(positions)

    return candidates, true_index

В функцию подается i-ый текст, разбитый на предложения, и список спенов для i-ого текста. В функции итерация по предложениям (разбиение на два) и итерация по спенам через очередь. Фиксируется длина текста во время итерации (text_len), а также каждый раз при разбиении на два предложения подсчитывается длина конкретного кусочка (curr_len).





In [ ]:
import heapq as hq

Еще одна проблема, которую я не учла при первичной своей предобработке: нужно было сделать булевый флажок катафора; соответственно сейчас необходимо, чтобы, если вдруг вторым линейным спеном оказывается ИГ, окно было расширено (не только в левую сторону антецедент, но и в правую). То есть для катафоры я просто запускаю поиск подходящих ИГ от всего текста.

In [ ]:
def create_new_dict(sentencized_text, span_list, text_pos, idx=0):
    dictionary = {}
    hq.heapify(span_list)
    max_length = len(sentencized_text)
    text_len = 0

    for pos in range(len(sentencized_text)):
        # print(pos, text_len)
        if pos + 1 == max_length:
            break

        new_text = ''.join(str(x) for x in [sentencized_text[pos], sentencized_text[pos+1]])
        first_sent_len = len(sentencized_text[pos])
        # print(new_text)
        curr_len = len(new_text)
        check = []
        while span_list:
            cataphora = False
            f = hq.heappop(span_list) # смотрим первый спен

            if f in ill_texts[text_pos]:
                continue

            if text_len+curr_len < f[1][1] or f[0][0] < text_len:
                hq.heappush(span_list, f)
                # если индекс конца второго элемента больше
                # то брейкаемся
                break

            corr_feats = None
            new_pair = f


            if new_pair[0][0] > text_len + first_sent_len and pos+1 != max_length-1:
                # если анафорическая пара в рамках одного (второго по счету) предложении
                # то надо включить его в оба разбиения (и там, где оно второе, и там где первое)
                check.append(new_pair)

            beg_ind_1, end_ind_1 = new_pair[0][0]-text_len, new_pair[0][1]-text_len
            beg_ind_2, end_ind_2 = new_pair[1][0]-text_len, new_pair[1][1]-text_len

            word_1 = new_text[beg_ind_1:end_ind_1]
            word_2 = new_text[beg_ind_2:end_ind_2]

            morph_an = tagger(word_2)


            if len(morph_an) == 1 and (morph_an[0].tag_ in ['PRON', 'DET', 'SCONJ']):
                  # анафорические местоимения тут, кажется, должны быть все одноразмерными
                  # если нет, то должен сработать последний else
                  corr_feats = {'Gender': morph_an[0].morph.get('Gender') or None,
                                  'Number': morph_an[0].morph.get('Number') or None,
                                  'Person': morph_an[0].morph.get('Person') or None}
                  true_cand_span = [beg_ind_1, end_ind_1]
                  true_an_span = [beg_ind_2, end_ind_2]
            else:
                morph_an = tagger(word_1)
                if len(morph_an) == 1 and (morph_an[0].tag_ in ['PRON', 'DET', 'SCONJ']):
                    # с катафорой то же самое
                    cataphora = True
                    corr_feats = {'Gender': morph_an[0].morph.get('Gender') or None,
                                  'Number': morph_an[0].morph.get('Number') or None,
                                  'Person': morph_an[0].morph.get('Person') or None}
                    true_cand_span = [beg_ind_2, end_ind_2]
                    true_an_span = [beg_ind_1, end_ind_1]
                else:

                    print(f"Warning: anaphora's length > 1: text pos = {text_pos}, word = {morph_an}, span = {beg_ind_2, end_ind_2}")
                    print(f'Second word = {word_2}')
                    print(len(tagger(word_2)), tagger(word_2)[0].tag_)
                    true_cand_span = None
                    true_an_span = None

            # для анафорического местоимения записывается словарь его морфологических характеристик
            # for word in morph_an:

            #     if word.tag_ in ['PRON', 'DET']:
            #         corr_feats = {'Gender': word.morph.get('Gender') or None,
            #                       'Number': word.morph.get('Number') or None,
            #                       'Person': word.morph.get('Person') or None}
            #     else:
            #         print(beg_ind_2, end_ind_2, word)

            candidates = None
            if corr_feats is not None:
                # получения словаря всех возможных антецедентов с их спенами и индекс правильного антецедента
                if cataphora:
                    candidates, true_index = morph_analysis(new_text, corr_feats, true_cand_span)
                else:
                    candidates, true_index = morph_analysis(new_text[:end_ind_2], corr_feats, true_cand_span)
            else:
                  print(f'Warning: no candidates, text pos = {text_pos}')

            # print('Записываю в словарь')
            if candidates is not None:
                dictionary[idx] = {'TwoSents': new_text,
                                  'AnaphPos': true_an_span,
                                  'AllAnts': candidates,
                                  'IndListAnt': true_index}
            else:
                print(f'Warning: no candidates, text pos = {text_pos}')

            idx += 1

        for s in check:
              # print('pushing it back')
              # print(pos, max_length)
              hq.heappush(span_list, s)

        check = []

        text_len += len(sentencized_text[pos])

    if span_list:
        print(text_pos, 'Warning: deque is not empty', span_list)

    return dictionary

In [ ]:
def restore_data(name='df_csv', labels=['Text', 'Span']):
    new_df = pd.read_csv(name, index_col=0)
    # new_df.drop(index=['Unnamed: 0'])
    for l in labels:
        new_df[l] = new_df[l].apply(lambda x: eval(x))
    # new_df['Span'] = new_df['Span'].apply(lambda x: eval(x))
    return new_df

In [ ]:
new_df = restore_data('df_csv')

In [ ]:
new_df

,Text,Span
0,"[""Чудовищно и необъяснимо"": Астахов прокоммент...","[[[1146, 1153], [1163, 1167]], [[1290, 1301], ..."
1,[Каждый десятый нотариус в Москве получил мест...,"[[[3966, 3981], [4109, 4111]], [[72, 106], [11..."
2,"[На ""Меридианах Тихого"" победил японский фильм...","[[[71, 123], [301, 304]], [[146, 235], [237, 2..."
3,[К психологам за помощью обратились 120 бывших...,"[[[152, 169], [179, 181]], [[794, 814], [817, ..."
4,[Роскомнадзор оштрафует Google за выдачу ссыло...,"[[[1867, 1879], [2200, 2203]], [[1419, 1465], ..."
...,...,...
3071,"[Аркадий Бабченко: ""Вы зачем Пашу убили, уроды...","[[[0, 16], [50, 51]], [[291, 300], [350, 351]]..."
3072,"[Доллар подрос на 17 копеек, евро - на 7., Ср...","[[[41, 205], [217, 225]], [[474, 492], [494, 4..."
3073,[Хоккеист Алексей Яшин продает свой особняк в ...,"[[[0, 21], [30, 34]], [[171, 178], [207, 211]]..."
3074,[Французская полиция задержала в Лилле группу ...,"[[[1213, 1243], [1245, 1252]], [[782, 799], [8..."


In [ ]:
create_new_dict(new_df.loc[104]['Text'], new_df.loc[104]['Span'], 104)

{0: {'TwoSents': '  Однако члены комитетов по науке сената и палаты представителей собираются подробно разобраться с обстоятельствами катастрофы и проанализировать в целом программу NASA "Space shuttle". Более того, они намереваются заглянуть гораздо глубже в планы NASA по изучению космического пространства на перспективу.',
  'AnaphPos': [198, 201],
  'AllAnts': [[9, 64]],
  'IndListAnt': 0},
 1: {'TwoSents': ' Руководитель американской правительственной комиссии по расследованию обстоятельств катастрофы шаттла Columbia Гарольд Геман уверен в том, что причины трагедии удастся установить. Об этом он заявил во вторник на пресс-конференции на мысе Канаверал, где расположены космодром и Космический центр имени Кеннеди, сообщает ИТАР-ТАСС.',
  'AnaphPos': [189, 191],
  'AllAnts': [[1, 125]],
  'IndListAnt': 0}}

In [ ]:
new_dict = create_new_dict(new_df.loc[13]['Text'], new_df.loc[13]['Span'], 13)

In [ ]:
new_dict

{0: {'TwoSents': ' Президент Российского футбольного союза Вячеслав Колосков считает, что будущее российского футбола за искусственными полями. Об этом он заявил на пресс-конференции в РИА "Новости", посвященной 5-й Международной выставке футбольной индустрии "Футбол Маркет" и второй специализированной выставке "Стадионное оборудование и современные покрытия на спортивных сооружениях".',
  'AnaphPos': [134, 136],
  'AllAnts': [[1, 58], [92, 99]],
  'IndListAnt': 0},
 1: {'TwoSents': '  По словам Колоскова, с учетом климата в России просто необходимо строить поля с искусственным газоном, и только тогда можно будет добиться прогресса в развитии национального футбола и, в частности, появлении новых футбольных дарований.  В связи с этим президент РФС отметил эксперимент генерального директора ОАО "Лужники", который уложил на главной арене страны искусственный газон.',
  'AnaphPos': [326, 333],
  'AllAnts': [[25, 31],
   [32, 39],
   [96, 103],
   [288, 324],
   [141, 150],
   [176, 183],
 

In [ ]:
test_df = pd.DataFrame(new_dict).T

In [ ]:
test_df.to_csv('test_csv')

In [ ]:
test_csv = restore_data('test_csv', labels=['AnaphPos', 'AllAnts'])

In [ ]:
new_df = restore_data()
pos = 0
final_data = []
for (idx, text, spans) in new_df.itertuples():
    if pos % 500 == 0:
        print(pos)
    dictionary = create_new_dict(text, spans, idx)
    final_data.append(dictionary)
    pos+=1

0
500
1000
1500
2000
2500


In [ ]:
# сохранение полученных предобработанных данных в json формат
with open("mydata.json", "w") as final:
    json.dump(final_data, final)

files.download('mydata.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Подготовка данных

In [ ]:
!curl -L -o data  "drive.google.com/uc?export=download&id=1S2dzrZd-6Nv3B7zuETODx4ead8769DkJ&confirm=yes"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 66.1M  100 66.1M    0     0  10.7M      0  0:00:06  0:00:06 --:--:-- 14.5M


In [ ]:
import json
import torch
from torch.utils.data.dataset import Dataset
from google.colab import files

In [ ]:
f = json.load(open('/content/data'))

In [ ]:
for x in f:
    for m in x.values():
      for pos, span in enumerate(m['AllAnts']):
          if span[0] == span[1]:
              if m['TwoSents'][span[0]-1:span[1]] != '-':
                  print(m['TwoSents'][span[0]-1:span[1]])
                  print(m)

In [ ]:
print(len(f))

2959


In [ ]:
train, dev, test = f[:2369], f[2369:2664], f[2664:]
print(len(train), len(dev), len(test))

2369 295 295


In [ ]:
def create_final_file(f):
    all_data = []
    for x in f:
        for m in x.values():
          idx = 0
          while m['TwoSents'][idx] == ' ':
              idx += 1

          m['TwoSents'] = m['TwoSents'][idx:]
          m['AnaphPos'][0] -= idx
          m['AnaphPos'][1] -= idx
          for pos, span in enumerate(m['AllAnts']):
              if span[0] == span[1]:
                  print(m)
              check = span[::]
              check[0] -= idx
              check[1] -= idx
              while check[0]-1 >= 0 and (m['TwoSents'][check[0]-1].isalpha() or m['TwoSents'][check[0]-1] == '-'):
                print(m['TwoSents'][check[0]-1])
                check[0] -= 1

              while len(m['TwoSents']) > check[1] and (m['TwoSents'][check[1]].isalpha() or m['TwoSents'][check[1]] == '-'):
                print(m['TwoSents'][check[1]])
                check[1] += 1

              m['AllAnts'][pos] = check
          all_data.append(m)
    return all_data

train_data = create_final_file(train)
dev_data = create_final_file(dev)
test_data = create_final_file(test)
print(len(train_data), len(dev_data), len(test_data), len(train_data)+len(dev_data)+len(test_data))

NameError: name 'train' is not defined

In [ ]:
def data_for_dataset(all_data):
    final_data = []
    for idx, elem in enumerate(all_data):
        for pos, span in enumerate(elem['AllAnts']):
            answer = dict()
            answer['TwoSents'] = elem['TwoSents']
            answer['AnaphPos'] = elem['AnaphPos']
            answer['Ant'] = span
            answer['IsAnt'] = int(pos == elem['IndListAnt'])
            answer['OrigPos'] = idx
            final_data.append(answer)
    return final_data

In [ ]:
train_d = data_for_dataset(train_data)
dev_d = data_for_dataset(dev_data)
test_d = data_for_dataset(test_data)
print(len(train_d), len(dev_d), len(test_d), len(train_d)+len(dev_d)+len(test_d))

In [ ]:
train_d[80422]

{'TwoSents': 'Солдата-срочника из Саратовской области Андрея Попова, который на 11 лет попал в рабство в Дагестане, сумел бежать, но по возвращении домой был арестован как дезертир, в четверг вечером выпустили из изолятора временного содержания. Однако свободы в полном смысле этого слова он не получил.',
 'AnaphPos': [275, 277],
 'Ant': [0, 16],
 'IsAnt': 0,
 'OrigPos': 27919}

In [ ]:
def download_data(data, filename):
    filename = filename + ".json"
    with open(filename, "w") as file:
        json.dump(data, file)
    files.download(filename)

download_data(train_d, 'train')
download_data(dev_d, 'dev')
download_data(test_d, 'test')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Создание датасета

In [ ]:
import json
import torch
from torch.utils.data.dataset import Dataset
from google.colab import files
from zipfile import ZipFile
from pathlib import Path

In [ ]:
!curl -L -o all_data "drive.google.com/uc?export=download&id=1XxyWtqSZ4xHTqv9U8HLHFKA1b7cS2a0h&confirm=yes"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 7172k  100 7172k    0     0  1160k      0  0:00:06  0:00:06 --:--:-- 1499k


In [ ]:
with ZipFile("/content/all_data", 'r') as file:
  file.extractall(path="/content")
file.close()

In [ ]:
train = json.load(open('train.json'))
dev = json.load(open('dev.json'))
test = json.load(open('test.json'))

In [ ]:
def convert_char_to_tokens(tokenization, char_pos, sent_idx=None):
  i = 0
  offset_mapping = tokenization["offset_mapping"]
  subtoken_start, subtoken_end = None, None
  approximate = None
  while subtoken_end is None:
    if i == len(offset_mapping):
        # print(sent_idx)
        if approximate is not None:
            subtoken_end = approximate
        else:
          subtoken_end = i

    if subtoken_start is None and (char_pos[0] == offset_mapping[i][0] or char_pos[0] == offset_mapping[i][1] or char_pos[0] == (offset_mapping[i][0] + offset_mapping[i][1])//2):
      subtoken_start = i
    if subtoken_end is None and (char_pos[1] == offset_mapping[i][0] or char_pos[1] == offset_mapping[i][1] or char_pos[1] == (offset_mapping[i][0] + offset_mapping[i][1])//2):
      subtoken_end = i+1
    if approximate is None and abs(char_pos[1]-offset_mapping[i][0]) <= 2:
        approximate = i+1
    i += 1
  return [subtoken_start, subtoken_end]

In [ ]:
class ARDataset(Dataset):

    def __init__(self, data, tokenizer, label=1, device = "cuda"):
        self.data = data
        self.tokenizer = tokenizer
        self.label = label
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        tokenization = self.tokenizer(item['TwoSents'], return_offsets_mapping=True)
        anaph_pos = convert_char_to_tokens(tokenization, item['AnaphPos'], sent_idx=index)
        ant_pos = convert_char_to_tokens(tokenization, item['Ant'], sent_idx=index)

        # if ant_pos[0] is None or ant_pos[1] is None:
        #     print(ant_pos, index)

        # if anaph_pos[0] is None or anaph_pos[1] is None:
        #     print(anaph_pos, index)

        answer = dict()
        answer['input_ids'] = torch.tensor(tokenization['input_ids'], dtype=torch.int64).to(self.device)
        answer['attention_mask'] = tokenization['attention_mask']
        answer['target_starts'] = torch.tensor(anaph_pos[0], dtype=torch.int64).to(self.device)
        answer['target_ends'] = torch.tensor(anaph_pos[1], dtype=torch.int64).to(self.device)
        answer['source_starts'] = torch.tensor(ant_pos[0], dtype=torch.int64).to(self.device)
        answer['source_ends'] = torch.tensor(ant_pos[1], dtype=torch.int64).to(self.device)
        answer['label'] = torch.tensor(int(item['IsAnt'] == self.label), dtype=torch.int64).to(self.device)
        answer['idx'] = torch.tensor(item['OrigPos'], dtype=torch.int64).to(self.device)

        return answer

In [ ]:
for line in train:
    if not isinstance(line['Ant'][0], int) or not isinstance(line['Ant'][1], int):
        print(line)

## Модель RoBERTa

In [ ]:
!pip install transformers==4.28.1

In [ ]:
!pip install accelerate -U

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AutoTokenizer, RobertaModel, RobertaPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.optimization import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
import numpy as np
import pandas as pd
import scipy

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

In [ ]:
tokenization = tokenizer(train[0]['TwoSents'], return_offsets_mapping=True)
print(tokenization)

{'input_ids': [1, 4603, 25021, 24814, 281, 33344, 9939, 263, 341, 22635, 30264, 38787, 10820, 18, 225, 705, 19896, 1662, 562, 39831, 5356, 9100, 22628, 833, 9395, 16, 25699, 16, 1206, 28742, 833, 16, 12089, 281, 4114, 18, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 3), (3, 8), (8, 13), (14, 15), (16, 24), (25, 32), (32, 33), (34, 36), (37, 44), (45, 51), (52, 61), (62, 69), (69, 70), (71, 71), (72, 73), (73, 75), (75, 79), (79, 82), (83, 88), (89, 96), (97, 105), (106, 113), (113, 116), (117, 124), (124, 125), (126, 142), (142, 143), (144, 149), (150, 152), (152, 155), (155, 156), (157, 166), (167, 168), (169, 175), (175, 176), (0, 0)]}


In [ ]:
train_dataset = ARDataset(train, tokenizer)
dev_dataset = ARDataset(dev, tokenizer)
test_dataset = ARDataset(test, tokenizer)

# train_dataset = ARDataset(train_d, tokenizer)
# dev_dataset = ARDataset(dev_d, tokenizer)
# test_dataset = ARDataset(test_d, tokenizer)

In [ ]:
for line in train_dataset:
    if line['ant_start'] == None:
        print(line)
    # print(line['ant_start'] == None)
    # break

In [ ]:
for key, value in train_dataset[0].items():
    print(key, value, sep="\n")

input_ids
tensor([    1,  4603, 25021, 24814,   281, 33344,  9939,   263,   341, 22635,
        30264, 38787, 10820,    18,   225,   705, 19896,  1662,   562, 39831,
         5356,  9100, 22628,   833,  9395,    16, 25699,    16,  1206, 28742,
          833,    16, 12089,   281,  4114,    18,     2], device='cuda:0')
attention_mask
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
anaph_start
tensor(28, device='cuda:0')
anaph_end
tensor(29, device='cuda:0')
ant_start
tensor(22, device='cuda:0')
ant_end
tensor(25, device='cuda:0')
label
tensor(1, device='cuda:0')
idx
tensor(0, device='cuda:0')


In [ ]:
class RobertaClassificationHead(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(4*config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, 2)

    def forward(self, first, second):
        vectors = torch.cat([first, second, first*second, torch.abs(first-second)], dim=-1)
        vectors = self.dense(vectors)
        vectors = torch.tanh(vectors)
        vectors = self.dropout(vectors)
        vectors = self.out_proj(vectors)
        return vectors


class RobertaForAnaphoraClassification(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = 2
        self.config = config
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.classifier = RobertaClassificationHead(config)
        self.post_init()

    def forward(self, input_ids, source_starts, source_ends, target_starts, target_ends, labels=None, return_dict=False, **kwargs):
        encoder_output = self.roberta(input_ids)['last_hidden_state']
        source_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, source_starts, source_ends)])
        target_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, target_starts, target_ends)])
        logits = self.classifier(source_encoder_output, target_encoder_output)
        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
        if not return_dict:
            return (logits,) if loss is None else (loss, logits)

        return SequenceClassifierOutput(loss=loss, logits=logits)

In [ ]:
roberta_model = RobertaForAnaphoraClassification.from_pretrained("ai-forever/ruRoberta-large", num_labels=2).to("cuda")

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaForAnaphoraClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForAnaphoraClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForAnaphoraClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForAnaphoraClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred_labels = np.argmax(logits, axis=-1)
    are_equal = (pred_labels == labels)
    TP = np.sum(are_equal * labels)
    FP = np.sum((1.0 - are_equal) * (1-labels))
    FN = np.sum((1.0 - are_equal) * (1-pred_labels))
    correct, total = np.sum(are_equal), len(labels)
    return {"Accuracy": 100 * correct / total, "P": 100 * TP / max(TP+FP, 1.0), "R": 100 * TP / max(TP+FN, 1.0), "F1": 100 * TP / max(TP+0.5*FN+0.5*FP, 1.0)}

In [ ]:
optimizer = AdamW(roberta_model.parameters(), lr=1e-5, weight_decay=0.01)

NameError: name 'roberta_model' is not defined

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=1,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=4, warmup_ratio=0.1,
                                  gradient_accumulation_steps=4,
                                  metric_for_best_model="Accuracy")

In [ ]:
test_train_dataset = ARDataset(train_d[:10], tokenizer)
test_dev_dataset = ARDataset(dev_d[:10], tokenizer)

In [ ]:
trainer = Trainer(
    model=roberta_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,P,R,F1
0,0.034100,0.037799,99.291962,98.988949,99.012230,99.000588


71040
8725
8722


TrainOutput(global_step=6345, training_loss=0.05252014194726005, metrics={'train_runtime': 7346.4871, 'train_samples_per_second': 13.819, 'train_steps_per_second': 0.864, 'total_flos': 1.7053065928156128e+16, 'train_loss': 0.05252014194726005, 'epoch': 1.0})

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="steps", save_strategy='steps',
                                  logging_strategy="steps", num_train_epochs=2,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=4, warmup_ratio=0.1,
                                  gradient_accumulation_steps=4,
                                  metric_for_best_model="Accuracy", eval_steps=500)

In [ ]:
trainer = Trainer(
    model=roberta_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,P,R,F1
500,0.510100,0.252738,89.579342,86.822086,83.207902,84.976582
1000,0.158400,0.100157,96.701374,93.384338,97.601129,95.446182
1500,0.105200,0.091256,97.634319,96.268657,97.083725,96.674473
2000,0.080300,0.089120,97.734277,98.655257,94.896519,96.739391
2500,0.067600,0.063414,98.417326,96.711132,98.894638,97.790698
3000,0.067700,0.059178,98.683882,97.648976,98.659454,98.151614
3500,0.050100,0.062633,98.725531,98.693276,97.695202,98.191703
4000,0.059900,0.049036,99.083715,98.275058,99.153340,98.712245
4500,0.041600,0.053629,99.008746,98.091692,99.129821,98.608024
5000,0.041100,0.040650,99.241983,99.056826,98.800564,98.928529


Step,Training Loss,Validation Loss,Accuracy,P,R,F1
500,0.510100,0.252738,89.579342,86.822086,83.207902,84.976582
1000,0.158400,0.100157,96.701374,93.384338,97.601129,95.446182
1500,0.105200,0.091256,97.634319,96.268657,97.083725,96.674473
2000,0.080300,0.089120,97.734277,98.655257,94.896519,96.739391
2500,0.067600,0.063414,98.417326,96.711132,98.894638,97.790698
3000,0.067700,0.059178,98.683882,97.648976,98.659454,98.151614
3500,0.050100,0.062633,98.725531,98.693276,97.695202,98.191703
4000,0.059900,0.049036,99.083715,98.275058,99.153340,98.712245
4500,0.041600,0.053629,99.008746,98.091692,99.129821,98.608024
5000,0.041100,0.040650,99.241983,99.056826,98.800564,98.928529


TrainOutput(global_step=12690, training_loss=0.05792300718050476, metrics={'train_runtime': 20589.1566, 'train_samples_per_second': 9.862, 'train_steps_per_second': 0.616, 'total_flos': 3.410665782541649e+16, 'train_loss': 0.05792300718050476, 'epoch': 2.0})

In [ ]:
test[1]

{'TwoSents': 'Работают следователи", - сказал уполномоченный по правам ребенка в Свердловской области Игорь Мороков, уточнив, что, по предварительным данным, медицинская помощь ребенку оказывалась в полном объеме. По его словам, девочка не из Свердловской области, в монастыре она находилась вместе с родителями.',
 'AnaphPos': [203, 206],
 'Ant': [163, 170],
 'IsAnt': 0,
 'OrigPos': 0}

In [ ]:
from collections import defaultdict

In [ ]:
def predict_with_trainer(trainer, test):
    same_sample = None
    count = defaultdict(int)
    correct, total = 0, 0
    for pos, elem in enumerate(test):
        if pos % 2500 == 0:
            print(pos)
        if same_sample is None:
            same_sample = []
            same_sample.append(elem)
            # это, наверно, не очень красиво
            # но пользуюсь тем, что знаю, что одинаковые предложения идут все подряд
            curr_idx = elem['OrigPos']
            continue
        if elem['OrigPos'] == curr_idx:
            same_sample.append(elem)

        else:
          if len(same_sample) > 1:
              test_dataset = ARDataset(same_sample, tokenizer)
              predictions = trainer.predict(test_dataset)

              pred = np.argmax(predictions[0][:,1])

              if same_sample[pred]['IsAnt'] == 1:
                  correct += 1
              total += 1
          count[len(same_sample)] += 1
          same_sample = []
          same_sample.append(elem)
          curr_idx = elem['OrigPos']
    if len(same_sample) > 1:
        test_dataset = ARDataset(same_sample, tokenizer)
        predictions = trainer.predict(test_dataset)

        pred = np.argmax(predictions[0][:,1])

        if same_sample[pred]['IsAnt'] == 1:
            correct += 1
        total += 1
        count[len(same_sample)] += 1
        same_sample = []
        same_sample.append(elem)
        curr_idx = elem['OrigPos']
    return correct, total

In [ ]:
correct, total count = predict_with_trainer(trainer, test)
print(correct, total)
print(count)

0


2500


5000


7500


10000


12500


3353 3375


In [ ]:
print(correct / total)

0.9937777777777778


## Проверка модели на собственных данных

In [ ]:
!curl -L -o new_test_data "drive.google.com/uc?export=download&id=1sv8bhS7B35KshRuQCfGs3NL07xX8emD3&confirm=yes"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 15560  100 15560    0     0   9045      0  0:00:01  0:00:01 --:--:-- 16931


In [ ]:
new_test_data = json.load(open('new_test_data'))
print(len(new_test_data))

25


In [ ]:
check_dataset = data_for_dataset(new_test_data)

In [ ]:
print(len(check_dataset))

62


In [ ]:
for pos, elem in enumerate(check_dataset):
  if elem['TwoSents'][elem['Ant'][0]:elem['Ant'][1]] == '':
      print(pos)
  if elem['TwoSents'][elem['AnaphPos'][0]:elem['AnaphPos'][1]] == '':
      print(pos)

In [ ]:
def my_predict(trainer, test):
    data = []
    same_sample = None
    count = defaultdict(int)
    correct, total = 0, 0
    for pos, elem in enumerate(test):
        if same_sample is None:
            same_sample = []
            same_sample.append(elem)
            curr_idx = elem['OrigPos']
            continue

        if elem['OrigPos'] == curr_idx:
            same_sample.append(elem)

        else:
          if len(same_sample) > 1:
              test_dataset = ARDataset(same_sample, tokenizer)
              predictions = trainer.predict(test_dataset)

              pred = np.argmax(predictions[0][:,1])

              if same_sample[pred]['IsAnt'] == 1:
                  print(same_sample[0]['TwoSents'], same_sample[pred]['Ant'], predictions[0])
                  correct += 1
              total += 1
          pred_span = same_sample[pred]['Ant']
          predicted_word = same_sample[0]['TwoSents'][pred_span[0]:pred_span[1]]
          data.append((same_sample[0]['TwoSents'], predicted_word, same_sample[0]['OrigPos']))
          count[len(same_sample)] += 1
          same_sample = []
          same_sample.append(elem)
          curr_idx = elem['OrigPos']
    if len(same_sample) > 1:
          test_dataset = ARDataset(same_sample, tokenizer)
          predictions = trainer.predict(test_dataset)

          pred = np.argmax(predictions[0][:,1])

          if same_sample[pred]['IsAnt'] == 1:
                  correct += 1
          total += 1
          pred_span = same_sample[pred]['Ant']
          predicted_word = same_sample[0]['TwoSents'][pred_span[0]:pred_span[1]]
          data.append((same_sample[0]['TwoSents'], predicted_word, same_sample[0]['OrigPos']))
          count[len(same_sample)] += 1
          same_sample = []
          same_sample.append(elem)
          curr_idx = elem['OrigPos']
    return correct, total, data

In [ ]:
correct, total, data = my_predict(trainer, check_dataset)

В заявлении суду представитель Министерства юстиции США заявил, что существовавшие ограничения больше не защищают общественные интересы в области конкуренции и их нужно отменить.  [83, 94] [[ 1.4413315 -1.7788885]
 [ 5.90744   -6.2595396]]


Процедить соус через мелкое сито, снять с его поверхности жир и выпарить на слабом огне, пока его не останется 1 стакан.  [10, 14] [[-5.154636   5.479924 ]
 [ 3.6659384 -4.21939  ]]


Шахов <...> стал пробираться к выходу, толпа, запрудившая вокзал, пронесла его, вместе с собою, до самого выхода.    [0, 5] [[-5.28604   6.209619]
 [ 5.015629 -5.353428]]


Мальчик при помощи спущенного рукава рубашки снял чайник с тагана и понес его в комнату.  [50, 56] [[ 5.6564417 -5.585459 ]
 [-5.800947   6.056895 ]
 [ 5.0814867 -5.2392883]]


Он оторвался от еды, отвлеченный голосом официанта: Маша пригубила бокал, искоса поглядывая на него. [0, 2] [[-5.114583   6.1284585]
 [ 5.3621593 -6.269212 ]
 [ 4.0587864 -4.433104 ]
 [ 5.349156  -5.627685 ]]


...

Но главное, когда пересылаешь коллегам письма контрагентов, они приходят к ним как от первоначального отправителя, и никто не спрашивает твоих комментариев, не дёргает лишний раз. [30, 38] [[-4.477755   6.0159483]
 [ 5.105855  -5.8791566]
 [ 4.9208603 -5.7983966]]


Лоси склонили рога, и Лис принялся начищать их песком.  [14, 18] [[-0.46101028  0.2689092 ]
 [-3.9084663   3.9039738 ]]


Путь вам в искусство, развивайте его дальше. [11, 20] [[ 5.2384734 -5.8563876]
 [-5.1032996  5.6429853]]


Тетя Вергиния попросила меня взять Ашхен с собой в деревню и подготовить ее за лето к переэкзаменовкам, иначе ее могли бы исключить из школы. [35, 40] [[-1.4121691  1.5549009]
 [-5.4196887  5.909562 ]
 [ 6.465554  -7.2799063]]


Литература учит глубже познавать жизнь и вызывает потребность изменить ее на пользу человека.  [33, 38] [[ 3.3796952 -3.4756277]
 [-5.485966   5.745478 ]
 [ 5.3972898 -5.6503468]]


In [ ]:
correct/total

0.64

In [ ]:
# легкие случаи
fast_check = [{'TwoSents': 'Когда Маша играла с куклой, к ней подошел ее брат.',
               'AnaphPos': [30, 33],
               'Ant': [6, 10],
               'IsAnt': 1,
               'OrigPos': 0
               },
                {'TwoSents': 'Когда Маша играла с куклой, к ней подошел ее брат.',
               'AnaphPos': [30, 33],
               'Ant': [20, 26],
               'IsAnt': 0,
                'OrigPos': 0
               },
              {'TwoSents': 'Ребята не хотели делать домашние задания по школьным предметам, поэтому они пошли гулять.',
               'AnaphPos': [72, 75],
               'Ant': [0, 6],
               'IsAnt': 1,
               'OrigPos': 1
               },
              {'TwoSents': 'Ребята не хотели делать домашние задания по школьным предметам, поэтому они пошли гулять.',
               'AnaphPos': [72, 75],
               'Ant': [33, 40],
               'IsAnt': 0,
               'OrigPos': 1
               },
              {'TwoSents': 'Ребята не хотели делать домашние задания по школьным предметам, поэтому они пошли гулять.',
               'AnaphPos': [72, 75],
               'Ant': [53, 62],
               'IsAnt': 0,
               'OrigPos': 1
               }
              ]

In [ ]:
correct, total, data = my_predict(trainer, fast_check)
correct/total

Когда Маша играла с куклой, к ней подошел ее брат. [6, 10] [[-5.4336705  6.1668363]
 [ 3.7345874 -4.8624554]]


1.0

In [ ]:
my_df = pd.DataFrame(data, columns=['Text', 'Prediction', 'Original data'])

In [ ]:
real_ants = []
for line in my_df.itertuples():
    true_idx = new_test_data[line[3]]['IndListAnt']
    real = new_test_data[line[3]]['AllAnts'][true_idx]
    real_ants.append(line[1][real[0]:real[1]])

In [ ]:
my_df['Real Ant'] = real_ants
my_df = my_df.drop(columns='Original data')
my_df['Correct'] = my_df['Prediction'] == my_df['Real Ant']
my_df

,Text,Prediction,Real Ant,Correct
0,"Участники дуэта, Нил Теннант и Крис Лоу, сообщ...",Крис Лоу,саундтрек,False
1,В заявлении суду представитель Министерства юс...,ограничения,ограничения,True
2,"Процедить соус через мелкое сито, снять с его ...",соус,соус,True
3,"Шахов <...> стал пробираться к выходу, толпа, ...",Шахов,Шахов,True
4,Мальчик при помощи спущенного рукава рубашки с...,чайник,чайник,True
5,"Франция чтит храбрых сынов России, отдавших жи...",России,Франция,False
6,"Он оторвался от еды, отвлеченный голосом офици...",Он,Он,True
7,"В течение всего дня обслуга, бдительно погляды...",обслуга,обслуга,True
8,"Легко и бесшумно добежав до края крыши, он ухв...",липы,ветвь,False
9,"Девушка на лошади проехала мимо Володи, а тот ...",Девушка,Девушка,True


## Берт

In [ ]:
!pip install transformers==4.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AutoTokenizer, BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.optimization import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
class BertaClassificationHead(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(4*config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, 2)

    def forward(self, first, second):
        vectors = torch.cat([first, second, first*second, torch.abs(first-second)], dim=-1)
        vectors = self.dense(vectors)
        vectors = torch.tanh(vectors)
        vectors = self.dropout(vectors)
        vectors = self.out_proj(vectors)
        return vectors

In [ ]:
class BertForAnaphoraClassification(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = 2
        self.config = config
        self.bert = BertModel(config, add_pooling_layer=False)
        self.classifier = BertaClassificationHead(config)
        self.post_init()

    def forward(self, input_ids, source_starts, source_ends, target_starts, target_ends, labels=None, return_dict=False, **kwargs):
        encoder_output = self.bert(input_ids)['last_hidden_state']
        source_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, source_starts, source_ends)])
        target_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, target_starts, target_ends)])
        logits = self.classifier(source_encoder_output, target_encoder_output)
        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
        if not return_dict:
            return (logits,) if loss is None else (loss, logits)

        return SequenceClassifierOutput(loss=loss, logits=logits)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
bert_model = BertForAnaphoraClassification.from_pretrained("ai-forever/ruBert-base", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForAnaphoraClassification: ['bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForAnaphoraClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForAnaphoraClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=1,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=8, warmup_ratio=0.1,
                                  gradient_accumulation_steps=8,
                                  metric_for_best_model="Accuracy")

In [ ]:
train_dataset = ARDataset(train, tokenizer)
dev_dataset = ARDataset(dev, tokenizer)
test_dataset = ARDataset(test, tokenizer)

In [ ]:
optimizer = AdamW(bert_model.parameters(), lr=1e-5, weight_decay=0.01)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=bert_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,P,R,F1
0,0.027100,0.058124,98.167430,96.580407,98.306679,97.435897


TrainOutput(global_step=1586, training_loss=0.02785266588194223, metrics={'train_runtime': 2086.5353, 'train_samples_per_second': 48.655, 'train_steps_per_second': 0.76, 'total_flos': 5439819739892736.0, 'train_loss': 0.02785266588194223, 'epoch': 1.0})

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=2,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=8, warmup_ratio=0.1,
                                  gradient_accumulation_steps=8,
                                  metric_for_best_model="Accuracy")

In [ ]:
trainer = Trainer(
    model=bert_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train(resume_from_checkpoint=True)

Epoch,Training Loss,Validation Loss,Accuracy,P,R,F1
1,0.021400,0.053206,98.433986,96.971798,98.659454,97.808347


TrainOutput(global_step=3172, training_loss=0.011210607174516175, metrics={'train_runtime': 2110.6341, 'train_samples_per_second': 96.2, 'train_steps_per_second': 1.503, 'total_flos': 1.0881212996621472e+16, 'train_loss': 0.011210607174516175, 'epoch': 2.0})

In [ ]:
correct, total = predict_with_trainer(trainer, test)
print(correct, total)

0


2500


5000


7500


10000


12500


3354 3375


In [ ]:
print(correct/total)

0.9937777777777778
